In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as dparser
import re
from datetime import datetime

In [ ]:
%matplotlib inline

In [ ]:
#kids = pd.read_excel('../Data/TEIS Exit Data for NSS.xlsx', sheet_name = 'exit')

In [ ]:
exit_notes = pd.read_excel('../Data/TEIS Exit Data for NSS.xlsx', sheet_name = 'Exit Notes')

In [ ]:
exit_notes

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
exit_notes = exit_notes.dropna(subset=['Exit Note'])   ##Drops rows with null values in "Exit Note"
exit_notes

In [ ]:
##Cell below drops specific columns

In [ ]:
ddid_exit = exit_notes.drop(columns = ['POE', 'DOB', 'Latest IFSP Date', 'Notification Date', 'Initial Eligibility Date'])
ddid_exit.head(12)

In [ ]:
#ddid_exit['Date'] = pd.to_datetime(ddid_exit['Exit Note'].str.extract(r'(?P<Date>\d+(?:\/\d+){2})', expand=False), errors='coerce')
#ddid_exit.head(50)

In [ ]:
test = ddid_exit.loc[181, 'Exit Note']    ##Locating row 181's Exit Note

In [ ]:
##cell below finds each instance of 'Reopened: ' and 'Exit Date: ' and extracts those dates. Does NOT put them into a column.

In [ ]:
match1 = re.findall(r'(Reopened:.*?\d{2}/\d{2}/\d{4}|Exit Date:.*?\d{2}/\d{2}/\d{4})', test)
match1

In [ ]:
##since re.findall did not place output into columns, the cell below uses str.split to do so. Notice that
##it is pulling all the text from "Exit Note"

In [ ]:
ddid_exit["Exit Note"].str.split("Reopened:", expand = True).head(30)

In [ ]:
##str.extractall pulls the reopened and exit dates but notice that 'reopened' and 'exit date' appear in the columns.

In [ ]:
ddid_date_test = ddid_exit['Exit Note'].str.extractall(r'(Reopened: \d+/\d+/\d+|Exit Date: \d+/\d+/\d+)').unstack()
ddid_date_test.head(50)

In [ ]:
##tried to use str.replace to take them out but had trouble figuring out why this was not working.

In [ ]:
replace_test = ddid_date_test[(0, 1)].str.replace('Reopened: ',' ')
replace_test

In [ ]:
##found what is called "look behind" or "look ahead", which is the ?<= that you see if front of 'reopened' and 'exit'.
## This took out the 'reopened' and 'exit date' in the columns. Note that these dates are still OBJECTS.
##Another thing using "look behind" did was create two capturing groups. This is why I have so many columns.

In [ ]:
ddid_date = ddid_exit['Exit Note'].str.extractall(r'(((?<=Reopened: )|(?<=Exit Date: ))\d{2}/\d{2}/\d{4})').reset_index(level = 1)
ddid_date.head(50)

In [ ]:
ddid_date.info()  #Capturing group created two levels.

In [ ]:
ddid_date = ddid_date.drop(columns = [1])

In [ ]:
ddid_date.info()

In [ ]:
ddid_all = exit_notes.merge(ddid_date, how = 'outer', left_index = True, right_index = True)
ddid_all.head(22)

In [ ]:
ddid_all.dtypes

In [ ]:
#ddid_all.iloc[:, 12].apply(pd.to_datetime, errors='coerce')

In [ ]:
ddid_all[0] = ddid_all[0].apply(pd.to_datetime)
ddid_all.info()

In [ ]:
mask = ddid_all[0] >= ddid_all['Initial IFSP Date']  ##this output is in true/false
test_mask = ddid_all[mask]  ##this calls the dataframe for everything that is true...pulls all the information
test_mask.info()

In [ ]:
mask2 = test_mask[0] < test_mask['Exit Date']
test_mask2 = test_mask[mask2]
test_mask2.info()

In [ ]:
test_mask2  #cannot unstack because of dumpster fire. also because it's not a multilayer index.

In [ ]:
ddid_pivot = test_mask2.reset_index().pivot_table(index = 'Child ID', columns = 'match', values = 0, aggfunc = 'first')
ddid_pivot  ##allows me to unstack 

In [ ]:
ddid_whole = exit_notes.merge(ddid_pivot, how = 'outer', left_on = 'Child ID', right_index = True)
ddid_whole.head(30)

In [ ]:
ddid_whole = ddid_whole.drop(columns = ['DOB', 'Notification Date', 'Initial Eligibility Date', 'Latest IFSP Date'])
ddid_whole

In [ ]:
#ddid_whole.head(45)  

In [ ]:
#ddid_date_test.head(30)

In [ ]:
#### next task - rename columns and start on aggregations

In [ ]:
ddid_whole = ddid_whole.rename(columns = {0.0 : 'Reopened_1', 1.0 : 'Exit_1', 2.0 : 'Reopened_2', 3.0 : 'Exit_2', 4.0 : 'Reopened_3', 5.0 : 'Exit_3', 6.0 : 'Reopened_4', 7.0 : 'Exit_4', 8.0 : 'Reopened_5', 9.0 : 'Exit_5', 10.0 : 'Reopened_6', 11.0 : 'Exit_6'})
ddid_whole.head(30)

In [ ]:
ddid_whole['days_1'] = ddid_whole['Reopened_1'] - ddid_whole['Exit_1']
ddid_whole.head(30)

In [ ]:
ddid_whole['days_2'] = ddid_whole['Reopened_2'] - ddid_whole['Exit_2']
ddid_whole.head(30)

In [ ]:
ddid_whole['days_3'] = ddid_whole['Reopened_3'] - ddid_whole['Exit_3']
ddid_whole.head(30)

In [ ]:
ddid_whole['days_4'] = ddid_whole['Reopened_4'] - ddid_whole['Exit_4']
ddid_whole.head(30)

In [ ]:
ddid_whole['days_5'] = ddid_whole['Reopened_5'] - ddid_whole['Exit_5']
ddid_whole.head(30)

In [ ]:
ddid_whole['days_6'] = ddid_whole['Reopened_6'] - ddid_whole['Exit_6']
ddid_whole.head(30)

In [ ]:
ddid_whole['total_gap_days'] = ddid_whole.iloc[:, -6:].sum(axis=1)
ddid_whole.head(50)

In [ ]:
ddid_whole['total_gap_days'] = ddid_whole['total_gap_days'].dt.days

In [ ]:
ddid_whole.info()

In [ ]:
ddid_whole['total_correct_days'] = ddid_whole['Days from I-IFSP to Exit'] - ddid_whole['total_gap_days']

In [ ]:
ddid_whole.head(50)

In [ ]:
ddid_whole.describe()

In [ ]:
ddid_whole.info()

In [ ]:
#ddid_whole.to_excel('Clean_Exit_Data.xlsx', sheet_name = 'clean_exit_data', index = False)

In [ ]:
#ddid_whole['total_days'] = ddid_whole['Days from I-IFSP to Exit'] - ddid_whole['total_days'].dt.days

In [ ]:
#ddid_whole['acutal_days'] = ddid_whole['Days from I-IFSP to Exit'] - ddid_whole['total_days']

In [ ]:
#ddid_whole.head(50)

FOR LOOP EXAMPLE  --- inconsistencies within data using this loop


for index, row in ddid_whole.iterrows():
    total_days = pd.Timestamp.today() - pd.Timestamp.today()
    days1 = row.Reopened_1 - row.Exit_1
    if (~pd.isnull(days1))!= -2:
        print(pd.isnull(days1))
        total_days += days1
    days2 = row.Reopened_2 - row.Exit_2
    if ~(pd.isnull(days2))!= -2:
        total_days += days2
    days3 = row.Reopened_3 - row.Exit_3
    if ~(pd.isnull(days3))!= -2:
        total_days += days3
    days4 = row.Reopened_4 - row.Exit_4
    if ~(pd.isnull(days4))!= -2:
        total_days += days4
    days5 = row.Reopened_5 - row.Exit_5
    if ~(pd.isnull(days5))!= -2:
        total_days += days5
    days6 = row.Reopened_6 - row['Exit Date']
    if ~(pd.isnull(days6))!= -2:
        total_days += days6
    ddid_whole.at[index,'total_days'] = total_days

In [ ]:
#ddid_whole['Actual_Days'] = ddid_whole['total_days'] - ddid_whole['Days from I-IFSP to Exit']

In [ ]:
#ddid_subset = ddid_all.loc[ddid_all['Exit Note'].str.contains(r'(.*Reopened:.*)')]

In [ ]:
##ddid_date = ddid_date.drop(columns=[(1, 0),(1, 1),(1, 2),(1, 3),(1, 4),(1, 5),(1, 6),(1, 7),(1, 8),(1, 9),(1, 10),(1, 11)])
##ddid_date.head()

In [ ]:
#ddid_date = ddid_date.rename(columns = {0: 'Reopened_1', 1: 'Exit_1', 2: 'Reopened_2', 3: 'Exit_2', 4: 'Reopened_3', 5: 'Exit_3', 6: 'Reopened_4', 
                                        7: 'Exit_4', 8: 'Reopened_5', 9:'Exit_5', 10: 'Reopened_6', 11: 'Exit_6'})
#ddid_date.head()

In [ ]:
#ddid_date.columns = ddid_date.columns.droplevel()  #Took out the the extra "Reopened_1" or the first level

In [ ]:
#ddid_date.head(50)

In [ ]:
#ddid_date.info()

In [ ]:
#ddid_date = ddid_date.iloc[:, 0:12].apply(pd.to_datetime, errors='coerce')


In [ ]:
#(ddid_date['Reopened_1'] - ddid_date['Exit_1']) 

In [ ]:
#ddid_all = ddid_exit.join(ddid_date, how = 'outer')
#ddid_all

In [ ]:
#ddid_subset = ddid_all.loc[ddid_all['Exit Note'].str.contains(r'(.*Reopened:.*)')]